# Construct and Print Office Hierarchy

**Imports for basic requirements:**

In [1]:
import json
import uuid
import copy

**Self defined tree structure:**

In [2]:
(_ADD, _DELETE, _INSERT) = range(3)
(_ROOT, _DEPTH, _WIDTH) = range(3)

class Node:

    def __init__(self, name, identifier=None, expanded=True):
        self.__identifier = (str(uuid.uuid1()) if identifier is None else str(identifier))
        self.name = name
        self.expanded = expanded
        self.__bpointer = None
        self.__fpointer = []

    @property
    def identifier(self):
        return self.__identifier

    @property
    def bpointer(self):
        return self.__bpointer    # parent (manager)

    @bpointer.setter
    def bpointer(self, value):
        if value is not None:
            self.__bpointer = value

    @property
    def fpointer(self):
        return self.__fpointer    # children (employees)

    def update_fpointer(self, identifier, mode=_ADD):
        if mode is _ADD:
            self.__fpointer.append(identifier)
        elif mode is _DELETE:
            self.__fpointer.remove(identifier)
        elif mode is _INSERT:
            self.__fpointer = [identifier]

class Tree:

    def __init__(self):
        self.nodes = []

    def get_index(self, position):
        for index, node in enumerate(self.nodes):
            if node.identifier == position:
                break
        return index

    def create_node(self, name, identifier=None, parent=None):

        node = Node(name, identifier)
        self.nodes.append(node)
        self.__update_fpointer(parent, node.identifier, _ADD)
        node.bpointer = parent
        return node

    def show(self, position, level=_ROOT):
        queue = self[position].fpointer
        if level == _ROOT:
            print("{0}".format(self[position].name))
        else:
            print("\t"*level, "{0}".format(self[position].name))
        if self[position].expanded:
            level += 1
            for element in queue:
                self.show(element, level)  # recursive call
                
    def show_tofile(self, position, level=_ROOT):
        queue = self[position].fpointer
        if level == _ROOT:
            with open("output.txt", "a") as f:
                f.write("{0}".format(self[position].name) + "\n")
                f.close()
        else:
            with open("output.txt", "a") as f:
                f.write("\t"*level + "{0}".format(self[position].name) + "\n")
                f.close()
        if self[position].expanded:
            level += 1
            for element in queue:
                self.show_tofile(element, level)  # recursive call
    
    def show_alphabetically(self, position, level=_ROOT):
        queue = self[position].fpointer
        queue.sort()
        if level == _ROOT:
            print("{0}".format(self[position].name))
        else:
            print("\t"*level, "{0}".format(self[position].name))
        if self[position].expanded:
            level += 1
            for element in queue:
                self.show_alphabetically(element, level)  # recursive call
                
    def show_alphabetically_tofile(self, position, level=_ROOT):
        queue = self[position].fpointer
        queue.sort()
        if level == _ROOT:
            with open("output.txt", "a") as f:
                f.write("{0}".format(self[position].name) + "\n")
                f.close()
        else:
            with open("output.txt", "a") as f:
                f.write("\t"*level + "{0}".format(self[position].name) + "\n")
                f.close()
        if self[position].expanded:
            level += 1
            for element in queue:
                self.show_alphabetically_tofile(element, level)  # recursive call    

    def __update_fpointer(self, position, identifier, mode):
        if position is None:
            return
        else:
            self[position].update_fpointer(identifier, mode)

    def __update_bpointer(self, position, identifier):
        self[position].bpointer = identifier
        
    def __getitem__(self, key):
        return self.nodes[self.get_index(key)]
    
    def __setitem__(self, key, item):
        self.nodes[self.get_index(key)] = item

    def __len__(self):
        return len(self.nodes)

    def __contains__(self, identifier):
        return [node.identifier for node in self.nodes if node.identifier is identifier]

## Processing a given office hierarchy from input.txt

In [3]:
with open("./employees.json", "r") as read_file:  
    employees = json.load(read_file)

In [4]:
# calculate total salary
salary_sum = 0
for record in employees:
    salary_sum += record["salary"]

In [5]:
# construct a dictionary that maps employees' record to their id
record_map = {}
for employee in employees:
    key = employee["id"]
    value = employee
    record_map[key] = value
    
record_copy = copy.deepcopy(record_map)

In [6]:
# prepare
office_forest = {}
added = set()

# add all roots in added and forest's roots set
for idx, record in record_map.items():
    if record["manager"] == None:
        manager = record["first_name"]
        added.add(manager)
        
        tree = Tree()
        tree.create_node(manager, manager.lower())    # one of roots
        office_forest[manager] = tree
        
# whether tree contains value 'name'
def contains(tree, name):
    names = []
    nodes = tree.nodes
    
    for node in nodes:
        names.append(node.name)
    if name in names: 
        return True
    return False

In [7]:
# construct office hierachy
while record_copy:
    for key, value in record_copy.items():
        name = value["first_name"]    # current employee's name
        
        manager_idx = value["manager"]    # current employee's manager's id
        v = ""    # current employee's manager's name
        
        if manager_idx != None:    # current employee has a manager
            manager_record = record_map[manager_idx]
            manager_name = manager_record["first_name"]
            v = manager_name    # get the manager's name

        if v == "":
            record_copy.pop(key)
            break
        else:
            if v in added:
                for root, tree in office_forest.items():
                    if contains(tree, v):
                        tree_nodes = tree.nodes
                        root = tree_nodes[0].name
                        office_forest[root].create_node(name, name.lower(), parent=v.lower())
                added.add(name)
                record_copy.pop(key)
                break

In [8]:
# final print 
for root, tree in office_forest.items():
    tree.show(root.lower())
print("Total salary: {0}".format(salary_sum))

Jeff
	 Dave
		 Andy
		 Jason
		 Dan
		 Rick
		 Suzanne
Total salary: 590000


## Extra Credit 1: Sort employees alphabetically

In [9]:
for root, tree in sorted(office_forest.items()):
    tree.show_alphabetically(root.lower())
print("Total salary: {0}".format(salary_sum))

Jeff
	 Dave
		 Andy
		 Dan
		 Jason
		 Rick
		 Suzanne
Total salary: 590000


## Extra Credit 2: Unit tests 

In [10]:
import unittest
import filecmp

def clean_file(file_dir):
    with open(file_dir, 'r+') as f:
        f.truncate(0)
        f.close()

class TestOfficeHierarchy(unittest.TestCase):
    
    def test_salary_amount(self):
        self.assertEqual(salary_sum, 590000)
    
    def test_print_without_order(self):
        clean_file("output.txt")
        
        for root, tree in office_forest.items():
            tree.show_tofile(root.lower())
            
        self.assertTrue(filecmp.cmp("output.txt", "sample_output/test_print_without_order.txt"))

    def test_print_in_order(self):
        clean_file("output.txt")
        
        for root, tree in sorted(office_forest.items()):
            tree.show_alphabetically_tofile(root.lower())
            
        self.assertTrue(filecmp.cmp("output.txt", "sample_output/test_print_in_order.txt"))

    def test_print_forest(self):
        clean_file("output.txt")
        
        forest = copy.deepcopy(office_forest)
        
        tree = Tree()
        tree.create_node("Extra", "Extra")    #root
        
        forest["Extra"] = tree
        
        for root, tree in forest.items():
            tree.show_tofile(root.lower())
            
        self.assertTrue(filecmp.cmp("output.txt", "sample_output/test_print_forest.txt"))

    
    def test_print_forest_without_order(self):
        clean_file("output.txt")
        
        forest = copy.deepcopy(office_forest)
        
        tree = Tree()
        tree.create_node("Chole", "chole")    #root
        
        forest["Chole"] = tree
        
        for root, tree in forest.items():
            tree.show_tofile(root.lower())
            
        self.assertTrue(filecmp.cmp("output.txt", "sample_output/test_print_forest_without_order.txt"))

    def test_print_forest_in_order(self):
        clean_file("output.txt")
        
        forest = copy.deepcopy(office_forest)
        
        tree = Tree()
        tree.create_node("Chole", "chole")    #root
        tree.create_node("Mary", "mary", parent = "chole")
        tree.create_node("David", "david", parent = "mary")
        tree.create_node("Moses", "moses", parent = "chole")
        
        forest["Chole"] = tree
        
        for root, tree in sorted(forest.items()):
            tree.show_alphabetically_tofile(root.lower())
            
        self.assertTrue(filecmp.cmp("output.txt", "sample_output/test_print_forest_in_order.txt"))

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
    clean_file("output.txt")


......
----------------------------------------------------------------------
Ran 6 tests in 0.010s

OK


## Extra Credit 3: Automatic data validation

In [11]:
from jsonschema import validate

# Describe what kind of json you expect.
schema = {
    "type" : "object",
    "properties" : {
        "id" : {"type" : "integer"},
        "first_name" : {"type" : "string"},
        "manager" : {"type" : ["integer", "null"]},
        "salary" : {"type" : "number"},
    },
}


with open("./employees.json", "r") as read_file:  
    my_json = json.load(read_file)
    read_file.close()
# Validate will raise exception if given json is not
# what is described in schema.
for json in my_json:
    validate(instance=json, schema=schema)